<a href="https://colab.research.google.com/github/YogeshParihar1311/content_based_recommendor_system/blob/main/content_based_recommendation_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
# from autoviz.classify_method import data_cleaning_suggestions ,data_suggestions

# from pycaret  import regression
from sklearn.model_selection import cross_val_score

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yogeshparihar1311","key":"41646bae978e8cfc336f2cbe53f584d0"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download rounakbanik/the-movies-dataset

 92% 210M/228M [00:01<00:00, 167MB/s]
100% 228M/228M [00:01<00:00, 142MB/s]


In [ ]:
! unzip the-movies-dataset.zip

Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [ ]:
pd.set_option("display.max_columns",None)

# Feature Engineering on credits

In [ ]:
df = pd.read_csv("credits.csv")

In [ ]:
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
df['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [ ]:
df['cast_names'] = [", ".join([i['name'] for i in eval(j)]) for j in df['cast']]

In [ ]:
df['cast_names'][0]

'Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette'

In [ ]:
df['director_names'] = [[i['name'] for i in eval(j)][0:1] for j in df['crew']]

In [ ]:
# for j in df['crew']:
#     a = [i['name'] for i in eval(j)][0:2]
#     print(a)

In [ ]:
df['director_names_nan'] = df['director_names'].apply(lambda x: np.nan if len(x)==0 else x[0])

In [ ]:
df['director_names_nan'].isnull().sum()

771

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop(['cast','crew','director_names'],axis=1,inplace=True)

In [ ]:
df.head()

,id,cast_names,director_names_nan
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Larry J. Franco
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Alan Silvestri


In [ ]:
cleaned_credits = df

In [ ]:
cleaned_credits.head()

,id,cast_names,director_names_nan
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Larry J. Franco
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Alan Silvestri


In [ ]:
cleaned_credits.shape

(44705, 3)

# Feature Engineering on movies

In [ ]:
df = pd.read_csv("movies_metadata.csv",usecols=['id','genres','original_language','overview','production_companies','production_countries','title'])

In [ ]:
df.shape

(45466, 7)

In [ ]:
df.head()

,genres,id,original_language,overview,production_companies,production_countries,title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",Father of the Bride Part II


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   genres                45466 non-null  object
 1   id                    45466 non-null  object
 2   original_language     45455 non-null  object
 3   overview              44512 non-null  object
 4   production_companies  45463 non-null  object
 5   production_countries  45463 non-null  object
 6   title                 45460 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


In [ ]:
df.duplicated().sum()

30

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.isnull().sum()/df.shape[0]*100

genres                  0.000000
id                      0.000000
original_language       0.024210
overview                2.099657
production_companies    0.006603
production_countries    0.006603
title                   0.013205
dtype: float64

In [ ]:
cleaned_credits['id'].nunique()

44661

In [ ]:
len(cleaned_credits.groupby(by='id'))

44661

In [ ]:
df['id'].nunique()

45436

In [ ]:
len(df.groupby(by='id'))

45436

In [ ]:
df['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [ ]:
df['genres'] = [", ".join([i['name'] for i in eval(j)]) for j in df['genres']]

In [ ]:
df.drop(['production_companies','production_countries'],axis=1,inplace=True)

In [ ]:
df.head()

,genres,id,original_language,overview,title
0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",Toy Story
1,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,Jumanji
2,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,Grumpier Old Men
3,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale
4,Comedy,11862,en,Just when George Banks has recovered from his ...,Father of the Bride Part II


In [ ]:
cleaned_movies = df

# Combine credit and movie dataset

In [ ]:
cleaned_credits.head()

,id,cast_names,director_names_nan
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Larry J. Franco
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Alan Silvestri


In [ ]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44705 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  44705 non-null  int64 
 1   cast_names          44705 non-null  object
 2   director_names_nan  44705 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [ ]:
cleaned_movies.head()

,genres,id,original_language,overview,title
0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",Toy Story
1,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,Jumanji
2,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,Grumpier Old Men
3,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale
4,Comedy,11862,en,Just when George Banks has recovered from his ...,Father of the Bride Part II


In [ ]:
cleaned_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45436 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   genres             45436 non-null  object
 1   id                 45436 non-null  object
 2   original_language  45425 non-null  object
 3   overview           44482 non-null  object
 4   title              45430 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [ ]:
cleaned_movies['id'] = [0 if '-' in i else int(i) for i in cleaned_movies['id']]

In [ ]:
df = pd.concat([cleaned_movies, cleaned_credits], axis=1, join='inner')

In [ ]:
df.isnull().sum()

genres                  0
id                      0
original_language      11
overview              923
title                   5
id                      0
cast_names              0
director_names_nan      0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(43727, 8)

In [ ]:
df.head()

,genres,id,original_language,overview,title,id,cast_names,director_names_nan
0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",Toy Story,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,Jumanji,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Larry J. Franco
2,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,Grumpier Old Men,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,Comedy,11862,en,Just when George Banks has recovered from his ...,Father of the Bride Part II,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Alan Silvestri


In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
df.head()

,genres,original_language,overview,title,cast_names,director_names_nan
0,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Jumanji,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Larry J. Franco
2,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Grumpier Old Men,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,Comedy,en,Just when George Banks has recovered from his ...,Father of the Bride Part II,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Alan Silvestri


In [ ]:
df['text'] = df['genres'] + ' ' + df['original_language'] + ' ' + df['overview'] + ' ' + df['cast_names'] + ' ' + df['director_names_nan']

In [ ]:
df.head()

,genres,original_language,overview,title,cast_names,director_names_nan,text
0,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",Toy Story,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,"Animation, Comedy, Family en Led by Woody, And..."
1,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Jumanji,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Larry J. Franco,"Adventure, Fantasy, Family en When siblings Ju..."
2,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Grumpier Old Men,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,"Romance, Comedy en A family wedding reignites ..."
3,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Comedy, Drama, Romance en Cheated on, mistreat..."
4,Comedy,en,Just when George Banks has recovered from his ...,Father of the Bride Part II,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Alan Silvestri,Comedy en Just when George Banks has recovered...


In [ ]:
df = df[['text','title']]

In [ ]:
df.head()

,text,title
0,"Animation, Comedy, Family en Led by Woody, And...",Toy Story
1,"Adventure, Fantasy, Family en When siblings Ju...",Jumanji
2,"Romance, Comedy en A family wedding reignites ...",Grumpier Old Men
3,"Comedy, Drama, Romance en Cheated on, mistreat...",Waiting to Exhale
4,Comedy en Just when George Banks has recovered...,Father of the Bride Part II


In [ ]:
# removing html tags
import re
def remove_tags(text):
    cleaned_txt = re.sub(re.compile('<.*?>'),'',text)
    return cleaned_txt

In [ ]:
df['text'] = df['text'].apply(remove_tags)

In [ ]:
# removing urls
def remove_urls(text):
    pattern = re.sub(re.compile(r'https?://\S+|www.\S+'),'',text)
    return pattern

In [ ]:
df['text'] = df['text'].apply(remove_urls)

In [ ]:
# remove punctuation
import string
exclude = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [ ]:
df['text'] = df['text'].apply(remove_punc)

In [ ]:
# remove chat words
chat_words = {
    "AFAIK":"As Far As I Know",
"AFK":"Away From Keyboard",
"ASAP":"As Soon As Possible",
"ATK":"At The Keyboard",
"ATM":"At The Moment",
"A3":"Anytime, Anywhere, Anyplace",
"BAK":"Back At Keyboard",
"BBL":"Be Back Later",
"BBS":"Be Back Soon",
"BFN":"Bye For Now",
"B4N":"Bye For Now",
"BRB":"Be Right Back",
"BRT":"Be Right There",
"BTW":"By The Way",
"B4":"Before",
"B4N":"Bye For Now",
"CU":"See You",
"CUL8R":"See You Later",
"CYA":"See You",
"FAQ":"Frequently Asked Questions",
"FC":"Fingers Crossed",
"FWIW":"For What It's Worth",
"FYI":"For Your Information",
"GAL":"Get A Life",
"GG":"Good Game",
"GN":"Good Night",
"GMTA":"Great Minds Think Alike",
"GR8":"Great!",
"G9":"Genius",
"IC":"I See",
"ICQ":"I Seek you",
"ILU":"I Love You",
"IMHO":"In My Honest/Humble Opinion",
"IMO":"In My Opinion",
"IOW":"In Other Words",
"IRL":"In Real Life",
"KISS":"Keep It Simple, Stupid",
"LDR":"Long Distance Relationship",
"LMAO":"Laugh My A.. Off",
"LOL":"Laughing Out Loud",
"LTNS":"Long Time No See",
"L8R":"Later",
"MTE":"My Thoughts Exactly",
"M8":"Mate",
"NRN":"No Reply Necessary",
"OIC":"Oh I See",
"PITA":"Pain In The A..",
"PRT":"Party",
"PRW":"Parents Are Watching",
"QPSA":"Que Pasa",
"ROFL":"Rolling On The Floor Laughing",
"ROFLOL":"Rolling On The Floor Laughing Out Loud",
"ROTFLMAO":"Rolling On The Floor Laughing My A.. Off",
"SK8":"Skate",
"STATS":"Your sex and age",
"ASL":"Age, Sex, Location",
"THX":"Thank You",
"TTFN":"Ta-Ta For Now!",
"TTYL":"Talk To You Later",
"U":"You",
"U2":"You Too",
"U4E":"Yours For Ever",
"WB":"Welcome Back",
"WTF":"What The F...",
"WTG":"Way To Go!",
"WUF":"Where Are You From?",
"W8":"Wait...",
"7K":"Sick:-D Laugher",
}

In [ ]:
chat_words = []
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
df['text'] = df['text'].apply(chat_conversion)

In [ ]:
# replace emoji
import emoji
def replace_emoji(text):
    return emoji.demojize(text)

In [ ]:
df['text'] = df['text'].apply(replace_emoji)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
# remove stop words
from nltk.corpus import stopwords
def remove_stopwords(text):
    new_text = ['' if word in stopwords.words('english') else word for word in text.split()]
    x = new_text[:]
    new_text.clear()
    return ' '.join(x)

In [ ]:
import time
start = time.process_time()
df['text'] = df['text'].apply(remove_stopwords)
print(time.process_time() - start)

518.84082383


In [ ]:
# stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
    return ' '.join([ps.stem(word) for word in text.split()])

In [ ]:
import time
start = time.process_time()
df['text'] = df['text'].apply(stem_words)
print(time.process_time() - start)

74.76344855100001


In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer(max_features=10000, stop_words='english')
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(df['text']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
simililarity = cosine_similarity(vectors)

In [ ]:
simi